In [15]:
from typing import Optional, Callable, List, Dict, Tuple
import os

import dgmc
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import torch
from torch import Tensor
import funcs
#import pytorch_lightning as pl
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric

%reload_ext autoreload
%autoreload 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [62]:
df_raw.shape

(793073, 22)

In [ ]:
dtypes = "string"

In [64]:
file_path = f"s3://drose-sandbox/sizmek_zync_cleaned_793k.csv"

df_raw = pd.read_csv(file_path, dtype="str")
print(f"Shape: {df_raw.shape[0]:,} Memory: {df_raw.memory_usage(deep=True).sum()/1e9:.2f}GB")

Shape: 793,073 Memory: 1.55GB


In [65]:
df_raw.head(1).T

,0
Unnamed: 0,0
account_id,19967
url,tvguide.com
city_code,5731371
state_code,OR
dma_code,820
country_code,US
user_id,5314600042548863
session_id,d1b6ae4f-7582-42d9-90bc-9b793bcc9ca7:162949223...
referrer,https://www.google.com/


In [67]:
sizmek_cols = ["user_id", "url", "account_id", "city_code",
               "state_code", "dma_code", "country_code", "sizmek_ip"]
zync_cols = ["session_id", "referrer", "client", "user_agent_platform",
             "user_agent_language", "user_agent_browser", "zync_ip", "zync_country",
             "zync_state", "zync_city", "zync_lat", "zync_long"]

df = df_raw[sizmek_cols + zync_cols]
print(df.shape)
#df = df[df[sizmek_cols + zync_cols].astype(bool)]  
#print(df.shape)
df = df.fillna("missing")
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df = df.reset_index(drop=True)
print(df.shape)

group_list = [e for e in (sizmek_cols + zync_cols) if e not in ("url", "referrer")]
df = df.groupby(group_list)[["url", "referrer"]].agg(lambda x: list(x))
df = df.reset_index()
print(df.shape)
df= df[df["user_id"].duplicated(keep=False)]
print(df.shape)
df = df.drop("user_id", axis=1)

sizmek_features = ["url", "account_id", "city_code", "state_code",
               "dma_code", "country_code", "sizmek_ip"]
zync_features = ["referrer", "client", "user_agent_platform", "user_agent_language", 
             "user_agent_browser", "zync_ip", "zync_country",
             "zync_state", "zync_city", "zync_lat", "zync_long"]

sizmek_df = df[sizmek_features]
zync_df = df[zync_features]

(793073, 20)
(793073, 20)
(793073, 20)
(793073, 20)
(44928, 20)
(30818, 20)


In [68]:
sizmek_df.head(3)

,url,account_id,city_code,state_code,dma_code,country_code,sizmek_ip
0,[foxnews.com],19967,4167147,FL,534,US,other
1,[https://www.dailymail.co.uk/tvshowbiz/article...,35927,4167147,FL,534,US,other
2,[buzzaboutbees.net],19967,5816861,WY,758,US,other


In [69]:
zync_df.head(3)

,referrer,client,user_agent_platform,user_agent_language,user_agent_browser,zync_ip,zync_country,zync_state,zync_city,zync_lat,zync_long
0,[https://www.giantfreakinrobot.com/ent/snake-e...,sizmek,android,10,chrome,other,US,FL,missing,28.6344,-81.6221
1,[https://www.giantfreakinrobot.com/ent/snake-e...,sizmek,android,10,chrome,other,US,FL,missing,28.6344,-81.6221
2,[https://www.wral.com/wake-county-assistant-pr...,sizmek,iphone,missing,safari,other,US,NC,Wake Forest,35.9825,-78.5376


# Create Dataset

In [73]:
class ZetaDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root: str, sizmek_df: pd.DataFrame, zync_df: pd.DataFrame, column: str, 
                 label: str, feature_cols=None, parse_url=False, expand_x=None, transform=None, 
                 pre_transform=None
        ):
        self.root = root
        self.sizmek_df = sizmek_df
        self.zync_df = zync_df
        self.column = column
        self.label = label
        self.feature_cols = feature_cols
        self.parse_url = parse_url
        self.expand_x = expand_x
        super(ZetaDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        print("Removing processed file. . .")
        os.remove(self.processed_paths[0])

    @property
    def raw_file_names(self) -> List[str]:
        return [
            "sizmek_bidstream_raw_20210625_10k.csv", 
            "zync_session_tracking_orc_20210625_10k.csv"
        ]

    @property
    def processed_file_names(self):
        return ["ZetaDataset.pt"]
    
    @property
    def edge_count_1(self):
        return zeta_data.data.edge_index1.shape[1]
    
    @property
    def edge_count_2(self):
        return zeta_data.data.edge_index2.shape[1]

    def download(self):
        pass

    def process(self):
        print("Creating Sizmek graph. . .")
        x1, edge_index1 = self.process_graph(
            self.sizmek_df, 
            self.column[0], 
            self.feature_cols["sizmek"]["categorical"])
        print("Creating Zync graph. . .")
        x2, edge_index2 = self.process_graph(
            self.zync_df, 
            self.column[1], 
            self.feature_cols["zync"]["categorical"], 
            expand_x=1126)

        train_y = self.process_y()
        test_y = self.process_y()

        data = Data(x1=x1, edge_index1=edge_index1, x2=x2,
                    edge_index2=edge_index2, train_y=train_y,
                    test_y=test_y)
        torch.save(self.collate([data]), self.processed_paths[0])

    def process_graph(self, df, column: str, feature_cols: List, expand_x: int=None):
        #df = pd.read_csv(file_path, low_memory=False)
        #df.columns = [i.split(".")[1] for i in df.columns]

        # parse URLs
        if self.parse_url == True and column in ["url", "referrer"]:
            print("Parsing URL")
            df[column] = df[column].apply(
                lambda x:funcs.urlparse(x).netloc if pd.notnull(x) else x
            )

        # Encode features
        print("Encoding features")
        feature_enc = OneHotEncoder(handle_unknown="ignore")
        features = pd.DataFrame(
            feature_enc.fit_transform(df[feature_cols]).toarray(), 
            columns=feature_enc.get_feature_names(feature_cols)
        )
        
        if expand_x is not None:
            print(f"Expanding X to {expand_x}")
            new_cols = [f"fake_{expand_x-i}" for i in range(expand_x - features.shape[1])][::-1]
            for col in new_cols:
                features[col] = 0
        x = torch.tensor(features.values, dtype=torch.float)
        
        #edges = funcs.connect_edges(df, column)
        #edge_index = torch.tensor(
        #    edges[['source','target']].T.values, dtype=torch.long
        #)
        print(f"Creating edge list on {column}")
        edges = funcs.connect_edges_2(df, column)
        edges = edges.sample(frac=.2)
        edge_index = torch.tensor(
            edges[["source", "target"]].T.values, dtype=torch.long
        )

        return x, edge_index

    def process_y(self) -> Tensor:
        y_1 = torch.tensor([range(0,2_000)])[0]
        y_2 = torch.tensor([range(0,2_000)])[0]
        train_y = torch.stack([y_1, y_2], dim=0)
        return train_y
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.edge_count_1:,}/{self.edge_count_2:,} edges)"
    
class SumEmbedding(object):
    def __call__(self, data):
        data.x1, data.x2 = data.x1.sum(dim=1), data.x2.sum(dim=1)
        return data

In [74]:
#sizmek_small = sizmek_df.sample(n=20_000, random_state=0)
#zync_small = zync_df.iloc[sizmek_small.index,:]

sizmek_small = sizmek_df.copy()
zync_small = zync_df.copy()

In [75]:
feature_cols = {
    "sizmek": {
        "categorical": [
            "account_id", "city_code", "state_code",
            "dma_code", "country_code", "sizmek_ip"
        ],
        "numeric": []
    },
    "zync": {
        "categorical": [
            "client", "user_agent_platform", "user_agent_language", 
            "user_agent_browser", "zync_ip", "zync_country",
            "zync_state", "zync_city"
        ],
        "numeric": ["zync_lat", "zync_long"]
    }
}

zeta_data = ZetaDataset(
    root="./data/",
    sizmek_df = sizmek_small,
    zync_df = zync_small,
    column=["url", "referrer"],
    label=["zeta_user_id", "client_id"],
    feature_cols=feature_cols,
    parse_url=False,
    transform=SumEmbedding()
)
zeta_data

Processing...
Creating Sizmek graph. . .
Encoding features
Creating edge list on url


KeyboardInterrupt: 

In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--dim', type=int, default=256)
parser.add_argument('--rnd_dim', type=int, default=32)
parser.add_argument('--num_layers', type=int, default=3)
parser.add_argument('--num_steps', type=int, default=10)
parser.add_argument('--k', type=int, default=10)
args = parser.parse_args("")


psi_1 = dgmc.models.RelCNN(zeta_data.data.x1.size(-1), args.dim, args.num_layers, batch_norm=False,
               cat=True, lin=True, dropout=0.1)
psi_2 = dgmc.models.RelCNN(args.rnd_dim, args.rnd_dim, args.num_layers, batch_norm=False,
               cat=True, lin=True, dropout=0.0)

psi_1
#psi_2

In [ ]:
model = dgmc.models.DGMC(psi_1, psi_2, num_steps=None, k=args.k).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
data = zeta_data.data

def train():
    model.train()
    optimizer.zero_grad()

    _, S_L = model(data.x1, data.edge_index1, None, None, data.x2,
                   data.edge_index2, None, None, data.train_y)

    loss = model.loss(S_L, data.train_y)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test():
    model.eval()

    _, S_L = model(data.x1, data.edge_index1, None, None, data.x2,
                   data.edge_index2, None, None)

    hits1 = model.acc(S_L, data.test_y)
    hits10 = model.hits_at_k(10, S_L, data.test_y)

    return hits1, hits10

In [ ]:
model.cuda()
zeta_data.data.cuda()

model.num_steps = 0
with torch.profiler.profile(
    schedule=torch.profiler.schedule(
        wait=2,
        warmup=2,
        active=6,
        repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler("./logs/gnn_v5"),
    with_stack=True
) as profiler:
    for epoch in range(1, 5000):
        print("Epoch: {}".format(epoch))
        if epoch == 101:
            print('Refine correspondence matrix...')
            model.num_steps = args.num_steps
            model.detach = True

        loss = train()
        profiler.step()
        
        if epoch % 5 == 0:
            hits1, hits10 = test()
            print((f"{epoch:03d}: Loss: {loss:.4f}, Hits@1: {hits1:.4f}, "
               f"Hits@10: {hits10:.4f}"))